# Data Collection
- Get list of 20 most visited cities.
- Create a dictionary with coordinates of 20 most visited cities.

In [1]:
import tqdm

### Get a list of the 20 most visited cities:

In [2]:
import requests
from scrapy import Selector
import json
import custom_functions as cf

Intel MKL WARNING: Support of Intel(R) Streaming SIMD Extensions 4.2 (Intel(R) SSE4.2) enabled only processors has been deprecated. Intel oneAPI Math Kernel Library 2025.0 will require Intel(R) Advanced Vector Extensions (Intel(R) AVX) instructions.
Intel MKL WARNING: Support of Intel(R) Streaming SIMD Extensions 4.2 (Intel(R) SSE4.2) enabled only processors has been deprecated. Intel oneAPI Math Kernel Library 2025.0 will require Intel(R) Advanced Vector Extensions (Intel(R) AVX) instructions.


In [3]:
cities_url = "https://travelness.com/most-visited-cities-in-the-world" # URL of the page with the list of cities

response = requests.get(cities_url)
sel = Selector(response)

cities = sel.xpath("//table//tr/td[2]/text()").getall()

### Use OpenStreetMaps API to convert the city names into coordinates

In [4]:
from geopy.geocoders import Nominatim

In [6]:
def geocode_city(city):
    """
    Geocode a single city using OpenStreetMap API.
    """
    geolocator = Nominatim(user_agent="my_geocoder")
    location = geolocator.geocode(city)
    return {"city": city, "latitude": location.latitude, "longitude": location.longitude}

def geocode_cities(city_list):
    """
    Geocode a list of cities using OpenStreetMap API.
    """
    geocoded_cities = [geocode_city(city) for city in city_list if geocode_city(city)]
    return geocoded_cities

# Geocode the list of cities
geocoded_cities = geocode_cities(cities)

In [7]:
# export geocoded cities to a json file
with open("../data/city_coordinates.json", "w") as f:
    json.dump(geocoded_cities, f)

### Use the open-meteo API to get weather data

In [8]:
# import relevant modules
import openmeteo_requests
import requests_cache
import pandas as pd
from retry_requests import retry

In [9]:
# Define our variables of interest as a list
daily_variables_of_interest = [
    "temperature_2m_max",
    "temperature_2m_min",
    "temperature_2m_mean",
    "daylight_duration",
    "sunshine_duration",
    "precipitation_sum",
    "rain_sum",
    "precipitation_hours",
]

# Create the API params dictionary
params = {
    "latitude": [city["latitude"] for city in geocoded_cities],
    "longitude": [city["longitude"] for city in geocoded_cities],
    "start_date": "1940-01-01",
    "end_date": "2023-12-31",
    "daily": daily_variables_of_interest,
}

In [10]:
# Setup the Open-Meteo API client with cache and retry on error
cache_session = requests_cache.CachedSession('.cache', expire_after = -1)
retry_session = retry(cache_session, retries = 5, backoff_factor = 0.2)
openmeteo = openmeteo_requests.Client(session = retry_session)

# Make sure all required weather variables are listed here
# The order of variables in hourly or daily is important to assign them correctly below
url = "https://archive-api.open-meteo.com/v1/archive"
responses = openmeteo.weather_api(url, params=params)

In [13]:
# List comprehension to create a list of dataframes
dataframes_list = [cf.process_response(response, geocoded_cities, i) for i, response in enumerate(responses)]

In [14]:
merged_df = pd.concat(dataframes_list, ignore_index=True)
merged_df

,date,city,temperature_2m_max,temperature_2m_min,temperature_2m_mean,daylight_duration,sunshine_duration,precipitation_sum,rain_sum,precipitation_hours
0,1940-01-01,Bangkok,27.061001,14.711000,20.571415,40775.191406,NaN,NaN,NaN,0.0
1,1940-01-02,Bangkok,26.961000,14.111000,20.248503,40786.789062,38087.828125,0.0,0.0,0.0
2,1940-01-03,Bangkok,26.261000,13.911000,19.719332,40799.343750,38101.910156,0.0,0.0,0.0
3,1940-01-04,Bangkok,27.961000,14.161000,20.523499,40812.839844,38117.046875,0.0,0.0,0.0
4,1940-01-05,Bangkok,28.661001,14.211000,21.300581,40827.265625,38133.210938,0.0,0.0,0.0
...,...,...,...,...,...,...,...,...,...,...
613615,2023-12-27,Hong Kong SAR,20.239000,13.739000,16.507751,38783.960938,18721.480469,2.1,2.1,5.0
613616,2023-12-28,Hong Kong SAR,21.639000,14.689000,17.126501,38795.007812,32057.478516,0.0,0.0,0.0
613617,2023-12-29,Hong Kong SAR,19.339001,14.389000,16.530668,38807.906250,30891.164062,0.0,0.0,0.0
613618,2023-12-30,Hong Kong SAR,22.939001,14.239000,18.253584,38822.621094,34756.941406,0.0,0.0,0.0
